In [1]:
using CSV, FileIO, DataFrames, Dates
using GMT

In [2]:
region="italy"
catalog="ingv"

df = CSV.read("./data/$region.csv", DataFrame);

In [3]:
# df = df[df.Magnitude .>= 3.0,:];

In [4]:
min_lon = minimum(df.Longitude)
max_lon = maximum(df.Longitude)
min_lat = minimum(df.Latitude)
max_lat = maximum(df.Latitude)
min_dep = minimum(df.Depth)
max_dep = maximum(df.Depth);

In [5]:
italy_coords = (min_lon,max_lon,min_lat,max_lat)
italy_coords_depth = (min_lon,max_lon,min_lat,max_lat,-max_dep,min_dep)

(6.6168, 18.513, 35.501, 47.083, -644.4, 0.0)

In [5]:
coast(region=italy_coords, proj=:merc, land=:gray, rivers="a", shore="0.25p", show=true)

In [6]:
marker_size = [2^x/100 for x in df.Magnitude];
# C_markers_depth = makecpt(cmap="red,blue", range=(minimum(df.Depth),maximum(df.Depth)),continuous=true,);
# C_markers_depth = makecpt(cmap=:seis, range=(minimum(df.Depth),maximum(df.Depth)),continuous=true,);
C_markers_mag = makecpt(cmap=:seis, range=(minimum(df.Magnitude),maximum(df.Magnitude)),continuous=true, inverse=true);

makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)


## 2D Plots

In [17]:
coast(region=italy_coords, 
        proj=:merc, land=:gray, rivers="a", shore="0.25p", alpha=50, 
        frame=(axes=:WSne,))
plot!(df.Longitude, df.Latitude, 
        markersize=marker_size, marker=:cc, markerline=:faint,
        cmap=C_markers_mag, zcolor=df.Magnitude, alpha=50, 
        show=true)

In [7]:
C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
relief_map = grdcut("@earth_relief_30s", region=italy_coords);

makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
GMT [WARNING]: Remote dataset given to a data processing module but no registration was specified - default to gridline registration (if available)
grdcut [WARNING]: (w - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: w reset from 6.6172 to 6.61666666667
grdcut [WARNING]: (e - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: e reset from 18.513 to 18.5166666667
grdcut [WARNING]: (s - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: s reset from 35.501 to 35.5


In [105]:
grdview(relief_map, proj=:merc, figsize=10, surftype=(image=100,), 
        cmap=C_map, zsize=1.5,view=(180,90) ) # shade=(azimuth=100, norm="e0.8"),
# colorbar!(pos=(outside=:MR,), shade=0.4, xaxis=(annot=:auto,), ylabel=:m, show=true)#view=(135,45),
colorbar!(pos=(anchor=:TC,length=(12.5,0.6), horizontal=true, offset=(0,1.0)),
xaxis=(annot=:auto,),frame=(ylabel=:m, offset=(0,1.0)),view=(180,90),savefig="./results/$region/test_test.png", show=true)


In [17]:
function scatter_2D(df, region, magnitude_threshold; z_control="Magnitude")

    df = df[df.Magnitude .>= magnitude_threshold,:];

    # Get region's coordinates
    min_lon = minimum(df.Longitude)
    max_lon = maximum(df.Longitude)
    min_lat = minimum(df.Latitude)
    max_lat = maximum(df.Latitude);

    # Create the map coordinates
    map_coords = (min_lon,max_lon,min_lat,max_lat)
    
    # Colormap for the region topography
    C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
    # Relief map of the region
    relief_map = grdcut("@earth_relief_30s", region=map_coords);

    
    # control marker size based on magnitude
    marker_size = [2^x/100 for x in df.Magnitude];

    # control marker color either by Magnitude or by Depth
    C_markers = makecpt(cmap=:seis, range=(minimum(df[!, z_control]),maximum(df[!, z_control])),continuous=true, inverse=true);
    colorbar_label = z_control
    zcolor_control = df[!, z_control]

    basemap(region=map_coords,frame=(axes=:WSne), proj=:merc)

    grdview!(relief_map, proj=:merc, axis=:none, surftype=(image=1000,), 
            cmap=C_map, zsize=1.5, alpha=40)

    plot!(df.Longitude, df.Latitude, 
            markersize=marker_size, marker=:cc, markerline=:faint,
            cmap=C_markers, zcolor=zcolor_control, alpha=60)

    colorbar!(pos=(outside=:MR, offset=(1.0,0)), shade=0.4, xaxis=(annot=:auto,), frame=(xlabel=colorbar_label,),par=(MAP_LABEL_OFFSET=0.8,), 
                savefig="./results/$region/$(region)_2D_mag_$(magnitude_threshold)_$(colorbar_label).png", show=true)

end

scatter_2D (generic function with 1 method)

In [18]:
function scatter_semi_3D(df, region, magnitude_threshold; z_control="Magnitude")

    df = df[df.Magnitude .>= magnitude_threshold,:];

    # Get region's coordinates
    min_lon = minimum(df.Longitude)
    max_lon = maximum(df.Longitude)
    min_lat = minimum(df.Latitude)
    max_lat = maximum(df.Latitude);

    # Create the map coordinates
    map_coords = (min_lon,max_lon,min_lat,max_lat)
    
    # Colormap for the region topography
    C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
    # Relief map of the region
    relief_map = grdcut("@earth_relief_30s", region=map_coords);

    
    # control marker size based on magnitude
    marker_size = [2^x/100 for x in df.Magnitude];

    # control marker color either by Magnitude or by Depth
    C_markers = makecpt(cmap=:seis, range=(minimum(df[!, z_control]),maximum(df[!, z_control])),continuous=true, inverse=true);
    zcolor_control = df[!, z_control]


    basemap(region=map_coords,frame=(axes=:SE), proj=:merc, view=(145,45))

    grdview!(relief_map, proj=:merc, axis=:none, surftype=(image=1000,), 
            cmap=C_map, zsize=1.5, alpha=40 , view=(145,45))

    plot!(df.Longitude, df.Latitude, 
            markersize=marker_size, marker=:cc, markerline=:faint,
            cmap=C_markers, zcolor=zcolor_control, alpha=60, view=(145,45))

    colorbar!(pos=(outside=:MR, offset=(1.5,0)), shade=0.4, xaxis=(annot=:auto,), frame=(xlabel=z_control,),par=(MAP_LABEL_OFFSET=0.8,), 
                view=(145,45), savefig="./results/$region/$(region)_semi3D_mag_$(magnitude_threshold)_$(z_control).png", show=true)

end

scatter_semi_3D (generic function with 1 method)

In [ ]:
for mag in [0.0,1.0,2.0,3.0,4.0]
    scatter_2D(df, "italy", mag; z_control="Magnitude")
    scatter_2D(df, "italy", mag; z_control="Depth")

    scatter_semi_3D(df, "italy", mag; z_control="Magnitude")
    scatter_semi_3D(df, "italy", mag; z_control="Depth")
    
end

In [ ]:
scatter_2D(df, "italy", 4.0; z_control="Magnitude")
scatter_2D(df, "italy", 4.0; z_control="Depth")

In [ ]:
scatter_semi_3D(df, "italy", 4.0; z_control="Magnitude")
scatter_semi_3D(df, "italy", 4.0; z_control="Depth")

## 3D Plots

In [276]:
marker_size = [2^x/100 for x in df.Magnitude];
# C_markers_depth = makecpt(cmap="red,blue", range=(minimum(df.Depth),maximum(df.Depth)),continuous=true,);
C_markers_depth = makecpt(cmap=:seis, range=(minimum(df.Depth),maximum(df.Depth)),continuous=true, inverse=true);
C_markers_mag = makecpt(cmap=:seis, range=(minimum(df.Magnitude),maximum(df.Magnitude)),continuous=true, inverse=true);

makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)


In [ ]:
C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
relief_map = grdcut("@earth_relief_30s", region=italy_coords);

In [110]:
italy_coords_depth

(6.6172, 18.513, 35.501, 47.075, -644.4, 0.0)

In [170]:
relief_map_depth = grdcut("@earth_relief_30s", region=italy_coords_depth);

GMT [WARNING]: Remote dataset given to a data processing module but no registration was specified - default to gridline registration (if available)
grdcut [WARNING]: (w - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: w reset from 6.6172 to 6.61666666667
grdcut [WARNING]: (e - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: e reset from 18.513 to 18.5166666667
grdcut [WARNING]: (s - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: s reset from 35.501 to 35.5


### Working but manually shifting the map

In [290]:
plot3d(df.Longitude, df.Latitude, -df.Depth,
frame="SEnwZ1+b xafg yafg zafg",proj=:merc,
markersize=marker_size, marker=:cc, # markerline=:faint,
cmap=C_markers_mag, zcolor=df.Magnitude,alpha=40,view=(135,20))

colorbar!(pos=(outside=:BC, offset=(0,1.5)), shade=0.4, xaxis=(annot=:auto,), frame=(xlabel="Magnitude",),par=(MAP_LABEL_OFFSET=0.8,),  view=(135,20))

grdview!(relief_map_depth, proj=:merc, surftype=(image=500,), 
cmap=C_map,zsize=0.1, alpha=30 ,yshift=6, view=(135,20), show=true)



# coast!(region=italy_coords, shorelines="1p,black", axis=:none,  proj=:merc, land="#ffe398",perspective=(135,25), alpha=50, show=true,yshift=5.4)

In [287]:
# https://www.earthinversion.com/utilities/how-to-plot-the-earthquake-data-on-three-dimensional-topographic-map/

depths = -1 .* maxdep .* ((df.Depth .- minimum(df.Depth) )./ (maximum(df.Depth)-minimum(df.Depth)) );

In [285]:
plot3d(df.Longitude, df.Latitude, depths,
# frame="SEnwZ1+b xafg yafg zafg",proj=:merc,
proj=:merc, frame="xa1f0.25 ya1f0.25 z2000+lmeters wSEnZ",
markersize=marker_size, marker=:cc, # markerline=:faint,
cmap=C_markers_depth, zcolor=df.Depth,alpha=40,view=(135,20))

grdview!(relief_map, region=(6.6172, 18.513, 35.501, 47.075,-maxdep,4000), axis=:none, proj=:merc, surftype=(image=500,), 
cmap=C_map, zsize=2, alpha=30, view=(135,20), show=true)

In [286]:
grdview(relief_map, region=(6.6172, 18.513, 35.501, 47.075,-maxdep,4000), frame="xa1f0.25 ya1f0.25 z2000+lmeters wSEnZ", proj=:merc, surftype=(image=500,), 
cmap=C_map, zsize=2, alpha=30, view=(135,20))

plot3d!(df.Longitude, df.Latitude, depths,
# frame="SEnwZ1+b xafg yafg zafg",proj=:merc,
proj=:merc, axis=:none,
markersize=marker_size, marker=:cc, # markerline=:faint,
cmap=C_markers_depth, zcolor=df.Depth,alpha=40,view=(135,20), show=true)

In [230]:
plot3d(df.Longitude, df.Latitude, -df.Depth,
frame="SEnwZ1+b xafg yafg zafg",proj=:merc,
markersize=marker_size, marker=:cc, # markerline=:faint,
cmap=C_markers_mag, zcolor=df.Magnitude,alpha=40,view=(135,20))

colorbar!(pos=(outside=:BC, offset=(0,1.5)), shade=0.4, xaxis=(annot=:auto,), frame=(xlabel="Magnitude",),par=(MAP_LABEL_OFFSET=0.8,),  view=(135,20))

grdview!(relief_map_depth,region=(6.6172, 18.513, 35.501, 47.075, -644000, 0.0), proj=:merc, surftype=(image=500,), 
cmap=C_map,zsize=0.1, alpha=30 ,yshift=6, view=(135,20), show=true)

In [ ]:
"a1Of1d WS"

scatter_2D (generic function with 2 methods)